***DeadLine - 29.09.2024 23:59***


Примечание:

Я убрал инструкцию, потому что мешает. 

С ноутбуком я работаю локально поэтому установка зависимостей ниже.

In [ ]:
packages = [
    "jupyter==1.0.0",
    "folium==0.17.0",
    "nltk==3.9.1",
    "gensim==4.3.3",
    "bokeh==3.6.0",
    "matplotlib==3.9.2",
    "seaborn==0.13.2",
    "scikit-learn==1.5.2",
]

with open("requirements.txt", "w") as file:
    file.write("\n".join(packages))

In [ ]:
!pip3 install -r requirements.txt > /dev/null

## Семинар 1: Веселье с векторными представлениями слов (2 балла)

Сегодня мы будем играть с векторными представлениями слов: обучим собственные небольшие эмбеддинги, загрузим готовую модель из gensim model zoo и используем её для визуализации текстового корпуса.

Вся работа будет происходить на наборе данных с векторными представлениями.

__Требования:__  `pip install --upgrade nltk gensim bokeh`, но только если вы запускаете код локально.

In [ ]:
!wget "https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1" -O ./quora.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(style="darkgrid")

np.random.seed(0)

In [ ]:
with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

__Токенизация:__ типичный первый шаг для задачи обработки естественного языка — это разделение исходных данных на слова. Текст, с которым мы работаем, находится в сыром формате: пунктация, смайлики и прочее

Давайте воспользуемся __`nltk`__ — библиотекой, которая выполняет многие задачи NLP, такие как токенизация, стемминг или определение частей речи.


In [ ]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

In [ ]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.

data_tok = list(map(lambda x: tokenizer.tokenize(x.lower()), data)) # map works better 
data_tok[:2]

In [ ]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"

assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"

is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)

assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [ ]:
print([' '.join(row) for row in data_tok[:2]])

__Векторы слов:__ как известно, существует множество способов для обучения векторных представлений слов. Есть Word2Vec и GloVe с разными целевыми функциями. Затем есть fastText, который использует n-gram модели для обучения векторных представлений слов.

Выбор огромен, так что давайте начнем с чего-то простого: __gensim__ – еще одна библиотека для обработки естественного языка, включающая множество моделей на основе векторов, включая word2vec.

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(data_tok,
                 vector_size=32, # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [ ]:
# now you can get word vectors !
model.get_vector('anything')

In [ ]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

### Использование предобученной модели

Заняло некоторое время, да? Теперь представьте себе обучение полноразмерных (100~300 измерений) векторных представлений слов на гигабайтах текста: статьи из Википедии или посты в Твиттере.

К счастью, сегодня вы можете получить предобученную модель векторных представлений слов всего за 2 строки кода (без смс и регистрации, обещаю).

In [ ]:
import gensim.downloader as gensim_api
model = gensim_api.load('glove-twitter-100')

In [ ]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

### Визуализация векторов слов

Один из способов понять, насколько хороши наши векторы, — это их визуализировать. Проблема в том, что эти векторы находятся в пространстве с размерностью 30+ измерений, а мы, люди, привыкли к 2-3 измерениям.

К счастью, мы, специалисты по машинному обучению, знаем о методах __снижения размерности__.

Давайте используем их, чтобы построить график для 1000 самых частотных слов.

In [ ]:
words = model.index_to_key[:1000]

print(words[::100])

In [ ]:
# for each word, compute it's vector with model
word_vectors = np.array(list(map(model.get_vector, words)))

In [ ]:
word_vectors

In [ ]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Линейная проекция: PCA

Самый простой метод линейного снижения размерности — это Principial Component Analysis.

В геометрических терминах PCA пытается найти оси, вдоль которых происходит наибольшая дисперсия. Это можно назвать "естественными" осями.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">

Под капотом алгоритм пытается разложить матрицу объект-признак $X$ на две меньшие матрицы: $W$ и $\hat W$, минимизируя _среднеквадратичную ошибку_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ — матрица объектов (**централизованная**);
- $W \in \mathbb{R}^{m \times d}$ — матрица прямого преобразования;
- $\hat{W} \in \mathbb{Р}^{d \times m}$ — матрица обратного преобразования;
- $n$ — количество объектов, $m$ — исходная размерность, $d$ — целевая размерность.

PCA работает, находя такие линейные проекции данных, которые сохраняют максимальное количество информации, измеряемое через дисперсию. Основная идея заключается в том, чтобы найти несколько новых осей (компонент), которые захватывают основную часть структуры данных.

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)

word_vectors_pca = (word_vectors_pca - word_vectors_pca.mean(axis=0)) / word_vectors_pca.std(axis=0) 

# and maybe MORE OF YOUR CODE here :)

In [ ]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

### Визуализация соседей с помощью t-SNE
PCA хорош для анализа данных, но это строго линейный метод, и он способен улавливать только грубую высокоуровневую структуру данных.

Если же мы хотим сосредоточиться на том, чтобы соседние точки данных оставались рядом, можно использовать t-SNE, который сам по себе является методом векторизации (embedding). Здесь можно __[узнать больше о t-SNE](https://distill.pub/2016/misread-tsne/)__.

t-SNE (t-distributed Stochastic Neighbor Embedding) — это метод нелинейного снижения размерности, который лучше всего подходит для визуализации высокоразмерных данных. Он работает, моделируя вероятность того, что объекты будут соседями в исходном пространстве, и затем пытается сохранить эти вероятности в проекции меньшей размерности. Основная идея t-SNE — это сохранение локальных структур данных, что делает его полезным для кластеризации и анализа сложных структур.

In [ ]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE(n_components=2).fit_transform(word_vectors)
word_tsne = (word_tsne - word_tsne.mean(axis=0)) / word_tsne.std(axis=0)

In [ ]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

### Визуализация фраз

Векторные представления слов также можно использовать для представления коротких фраз. Самый простой способ — взять __среднее значение__ векторов всех токенов в фразе с определёнными весами.

Этот приём полезен для понимания, с какими данными вы работаете: можно определить наличие выбросов, кластеров или других артефактов.

In [ ]:
model.has_index_for("hi"), model.has_index_for("hidddwa")

In [ ]:
def get_phrase_embedding(phrase: str, show_debug: bool = True) -> np.ndarray:
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    tokens = tokenizer.tokenize(phrase.lower())
    if show_debug:
        print(f"[get_phrase_embedding] Tokens before filter: {len(tokens)}")
    tokens_filtered = [token for token in tokens if model.has_index_for(token)]
    if show_debug:
        print(f"[get_phrase_embedding] Tokens after filter: {len(tokens_filtered)}")
    if len(tokens_filtered) == 0:
        return np.zeros(shape=model.vector_size)
    
    tokens_filtered = np.array(list(map(model.get_vector, tokens_filtered)))
    
    return tokens_filtered.mean(axis=0)



In [ ]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [ ]:
len(data)

In [ ]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::(len(data) // 1000)]

# compute vectors for chosen phrases
phrase_vectors = np.array(list(map(get_phrase_embedding, chosen_phrases))) # YOUR CODE

In [ ]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [ ]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = PCA(n_components=2).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0) 


In [ ]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Наконец, давайте создадим простой движок для поиска "похожих вопросов" с использованием векторных представлений фраз, которые мы создали.

In [ ]:
# compute vector embedding for all lines in data
data_vectors = np.array(list(map(lambda x: get_phrase_embedding(x, show_debug=False), data)))

In [ ]:
def find_nearest(query: str, k: int = 10, get_sim: bool = False):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    query_vector: np.ndarray = get_phrase_embedding(query)
    
    similarities: np.ndarray = model.cosine_similarities(query_vector, vectors_all=data_vectors)
    
    # `nan` handling, `nan` should be shifted into out of similarity scale to left side 
    similarities[np.isnan(similarities)] = -1.
    
    top_k_partitioned = np.argpartition(similarities, -k)[-k:]
    print("[find_nearest] top_k_partitioned.shape: ", top_k_partitioned.shape)
    
    top_k_indices = top_k_partitioned[np.argsort(similarities[top_k_partitioned])[::-1]]
    print("[find_nearest] top_k_indices.shape: ", top_k_indices.shape)
    print("="*100)
    print("[find_nearest]: Founded top k with sim score: ", [(data[i], similarities[i]) for i in top_k_indices])
    print("="*100)
    
    if get_sim:
        top_k_results = [(data[i], similarities[i]) for i in top_k_indices]
    else:
        top_k_results = [data[i] for i in top_k_indices if not np.isnan(similarities[i])]

    return top_k_results

In [ ]:
results = find_nearest(query="How do i enter the matrix?", k=10)
results

In [ ]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

##Multilingual Embedding-based Machine Translation (3 points)

**В этой ЛР** **<font color='red'>ТЫ</font>** создашь систему машинного перевода без использования параллельных корпусов, выравнивания, внимания, сверхкрутых рекуррентных нейронных сетей с 100500-уровневой глубиной и всех прочих наворочек

Но даже без параллельных корпусов эта система может быть достаточно хорошей (ну точно)

Для нашей системы мы выбираем два родственных славянских языка: украинский и русский.



### Почувствуй разницу!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Фрагмент списка Swadesh для некоторых славянских языков

Список Swadesh - это лексико-статистический материал. Он назван в честь американского лингвиста Морриса Свадеша и содержит базовую лексику. Этот список используется для определения подгрупп языков, их родства.

Таким образом, мы можем наблюдать некую инвариантность слов для разных славянских языков.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

## Data

In [ ]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

"Скачивать эмбеды надо тут:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

In [ ]:
# !unzip "cc.ru.300.vec.zip" > /dev/null

In [ ]:
# !unzip "cc.uk.300.vec.zip" > /dev/null

In [ ]:
!ls

Качаем эмбеды

In [ ]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [ ]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [ ]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

In [ ]:
uk_emb.most_similar([uk_emb["серпень"]])

In [ ]:
ru_emb.most_similar([uk_emb["серпень"]])

Построить словари для тренировочной и тестовой выгрузки (да, да, будет)

А вот и файлы!

[ukr-rus-test](https://disk.yandex.ru/i/fXtdDXKwgJloCw)

[ukr-rus-train](https://disk.yandex.ru/d/lUy0sQZ1XBCchQ)

In [ ]:
from typing import Tuple


def load_word_pairs(filename: str) -> Tuple[str, np.ndarray, np.ndarray]:
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if (uk not in uk_emb) or (ru not in ru_emb):
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [ ]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [ ]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping

Мы рассматриваем задачу обучения линейного преобразования, которое минимизирует различие между векторными представлениями слов в исходном и целевом языках.

Пусть $x_i \in \mathbb{R}^d$ — векторное представление слова на исходном языке, а $y_i \in \mathbb{R}^d$ — вектор его перевода на целевой язык. Наша цель — найти матрицу $W$, которая минимизирует евклидово расстояние между преобразованными векторами $Wx_i$ и их соответствующими $y_i$.

Очень похоже на линейную регрессию, не так ли?

$$W^* = \arg\min_W \sum_{i=1}^n ||Wx_i - y_i||_2^2,$$


В мифологии Прокруст был известен тем, что «подгонял» людей под размеры своей кровати, растягивая их или отрезая им ноги, чтобы они идеально подходили. Подобным образом, в данной задаче мы «подгоняем» пространство встраивания исходного языка к пространству целевого языка, пытаясь свести их различия к минимуму.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

Целевой язык - русский, исходный - украинский

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge

# YOUR CODE HERE
# mapping = Ridge(alpha=1.)
mapping = LinearRegression()
mapping.fit(X_train, Y_train)
W = mapping.coef_

In [ ]:
def evaluate_model(W, X_test, Y_test):
    transformed_vectors = X_test.dot(W.T)  # Преобразование векторов с помощью матрицы W
    error = np.linalg.norm(transformed_vectors - Y_test, axis=1).mean()  # Средняя ошибка
    return error

# Оценка модели
error = evaluate_model(W, X_test, Y_test)
print(f"Средняя ошибка на тестовых данных: {error}")

Давайте посмотрим на соседей вектора слова _«серпень»_ (_«август»_ по-русски) после линейного преобразования.

In [ ]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

Верный месяц всего на 9ом месте, хм...


В качестве меры качества мы будем использовать точность top@1, top@5 и top@10 (для каждого преобразованного украинского эмбеддинга мы подсчитываем, сколько правильных пар найдено в N ближайших соседях в пространстве русских эмбеддингов)

In [ ]:
from joblib import Parallel, delayed


def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = список пар правильных слов [(uk_word_0, ru_word_0), ...].
        mapped_vectors = список эмбедов после отображения из исходного пространства эмбедов в целевое пространство эмбедов
        topn = количество ближайших соседей в целевом пространстве эмбедов, из которых нужно выбрать
    """
    assert len(pairs) == len(mapped_vectors)

    def check_match(i):
        _, ru = pairs[i]
        most_similar = ru_emb.most_similar(mapped_vectors[i], topn=topn)
        return 1 if ru in [word for word, _ in most_similar] else 0 # Это быстрее чем цикл for и внутри смотреть if

    num_matches = sum(Parallel(n_jobs=-1)(delayed(check_match)(i) for i in range(len(pairs))))
    precision_val = num_matches / len(pairs)
    return precision_val


In [ ]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [ ]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [ ]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

In [ ]:
precision_top1

In [ ]:
precision_top5

## Making it better (orthogonal Procrustean problem)

Можно показать (см. оригинальную статью), что между семантическими пространствами должно быть ортогональным.

Мы можем ограничить преобразование $W$, чтобы оно было ортогональным. Тогда мы решим следующую задачу:

$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

$$I \text{- единичная матрица}$$.

Но ограничивать лосс в линейной регрессии как-то в падлу, поэтому мы можем найти оптимальное ортогональное преобразование с помощью разложения по сингулярным значениям (в чате тг я скидывал ссылки на краткое объяснение SVD). Оказывается, оптимальное преобразование $W^*$ может быть выражено через компоненты SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$.
$$W^*=UV^T$$

In [ ]:
def learn_transform(X_train, Y_train):
    """
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """

    M = X_train.T @ Y_train
    U, _, Vt = np.linalg.svd(M)
    W_star = U @ Vt
    return W_star

In [ ]:
W = learn_transform(X_train, Y_train)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

In [ ]:
svd_precision = precision(uk_ru_test, np.matmul(X_test, W))
svd_precision_top_5 = precision(uk_ru_test, np.matmul(X_test, W), 5)

assert svd_precision >= 0.653
assert svd_precision_top_5 >= 0.824

In [ ]:
svd_precision

In [ ]:
svd_precision_top_5

## UK-RU Translator

Теперь мы готовы сделать простой переводчик на основе слов: для каждого слова исходного языка в общем пространстве вкраплений мы находим ближайшее в языке перевода.

[fairy_tale.txt](https://disk.yandex.ru/d/4P9xWLfm-GnauQ) - качаем

In [ ]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [ ]:
def translate(sentence: str) -> str:
    """
    :args:
        sentence - предложение на украинском языке (str)
    :returns:
        перевод - предложение на русском языке (str)

    * найти украинские эмбеды для каждого слова в предложении
    * преобразовать вектор украинских эмбедов
    * найти ближайшее русское слово и заменить на него
    """
    translated_words = []
    for word in sentence.split():
        if word in uk_emb:
            uk_vector = uk_emb[word].reshape(1, -1)
            ru_vector = uk_vector @ W
            most_similar = ru_emb.most_similar(ru_vector, topn=1)
            translated_words.append(most_similar[0][0])
        else:
            translated_words.append(word)  # Сохраняем слово, если его нет в эмбеддингах
    return " ".join(translated_words)

In [ ]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [ ]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

Очень неплохо, но как можно еще улучшить результат?

## А теперь самое интересное?

### Почитать:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)
